#  नोटबुक 1: PP-OCR के लिए डेटा तैयार करना 📝

**उद्देश्य:** इस नोटबुक का लक्ष्य किसी भी मानक एनोटेशन प्रारूप से पैडलओसीआर (PaddleOCR) के लिए आवश्यक डिटेक्शन और रिकॉग्निशन लेबल प्रारूपों में डेटासेट को डाउनलोड, अनज़िप और परिवर्तित करना है। हम ट्रेनिंग और वैलिडेशन सेट भी बनाएंगे।

---

## चरण 1: परिवेश सेटअप

In [ ]:
# आवश्यक पुस्तकालयों को स्थापित करें
!pip install opencv-python-headless Pillow

---

## चरण 2: डेटासेट डाउनलोड और व्यवस्थित करें

हम प्रदर्शन के लिए एक छोटे, काल्पनिक डेटासेट का उपयोग करेंगे। वास्तविक परिदृश्य में, आप यहाँ COCO-Text या ICDAR जैसे डेटासेट को डाउनलोड और अनज़िप करेंगे।

In [ ]:
import os
import json
import random
from PIL import Image, ImageDraw, ImageFont

# डेटासेट के लिए निर्देशिकाएँ बनाएँ
os.makedirs('dataset/images', exist_ok=True)
os.makedirs('dataset/word_crops', exist_ok=True)

# एक काल्पनिक एनोटेशन फ़ाइल बनाएँ (जैसे, COCO-Text से)
annotations = {
    "img_01.png": [
        {"transcription": "PADDLE", "points": [[10, 15], [100, 16], [99, 50], [11, 49]]},
        {"transcription": "OCR", "points": [[120, 20], [180, 20], [180, 55], [120, 55]]}
    ],
    "img_02.png": [
        {"transcription": "HELLO", "points": [[25, 30], [150, 28], [151, 80], [26, 82]]}
    ]
}

# प्रदर्शन के लिए डमी छवियाँ बनाएँ
def create_dummy_image(text, path, size=(200, 100)):
    img = Image.new('RGB', size, color='white')
    draw = ImageDraw.Draw(img)
    try:
        font = ImageFont.truetype("arial.ttf", 15)
    except IOError:
        font = ImageFont.load_default()
    draw.text((10, 10), text, fill='black', font=font)
    img.save(path)

create_dummy_image("PADDLE OCR", "dataset/images/img_01.png")
create_dummy_image("HELLO", "dataset/images/img_02.png")

print("काल्पनिक डेटासेट बनाया गया।")

---

## चरण 3: डिटेक्शन लेबल में रूपांतरण

हम `det_label.txt` फ़ाइल बनाएंगे, जहाँ प्रत्येक पंक्ति में `image_path\tjson_string` होता है।

In [ ]:
det_labels = []
for img_name, ann_list in annotations.items():
    line = f"images/{img_name}\t{json.dumps(ann_list)}"
    det_labels.append(line)

# इसे एक फ़ाइल में लिखें
with open('dataset/det_label.txt', 'w') as f:
    f.write('\n'.join(det_labels))

print("dataset/det_label.txt बनाया गया:")
!cat dataset/det_label.txt

---

## चरण 4: रिकॉग्निशन लेबल में रूपांतरण

हम डिटेक्शन एनोटेशन से शब्द छवियों को क्रॉप करेंगे और `rec_label.txt` फ़ाइल बनाएंगे, जिसमें `crop_path\tlabel` होता है।

In [ ]:
rec_labels = []
crop_counter = 0

for img_name, ann_list in annotations.items():
    img = Image.open(f"dataset/images/{img_name}")
    for ann in ann_list:
        points = ann['points']
        transcription = ann['transcription']
        
        # सीमा बॉक्स प्राप्त करें और क्रॉप करें
        x_coords = [p[0] for p in points]
        y_coords = [p[1] for p in points]
        box = (min(x_coords), min(y_coords), max(x_coords), max(y_coords))
        cropped_img = img.crop(box)
        
        # क्रॉप की गई छवि को सहेजें
        crop_filename = f"word_crops/crop_{crop_counter:04d}.png"
        cropped_img.save(f"dataset/{crop_filename}")
        
        # रिकॉग्निशन लेबल में जोड़ें
        rec_labels.append(f"{crop_filename}\t{transcription}")
        crop_counter += 1

# रिकॉग्निशन लेबल फ़ाइल लिखें
with open('dataset/rec_label.txt', 'w') as f:
    f.write('\n'.join(rec_labels))
    
print("dataset/rec_label.txt बनाया गया:")
!cat dataset/rec_label.txt

---

## चरण 5: ट्रेनिंग/वैलिडेशन विभाजन बनाएँ

वास्तविक उपयोग के लिए, आपको अपने लेबल को अलग-अलग `train.txt` और `val.txt` फ़ाइलों में विभाजित करना चाहिए।

In [ ]:
def create_splits(label_file_path, train_ratio=0.8):
    with open(label_file_path, 'r') as f:
        lines = f.readlines()
    
    random.shuffle(lines)
    train_size = int(len(lines) * train_ratio)
    train_lines = lines[:train_size]
    val_lines = lines[train_size:]
    
    base_name = os.path.splitext(label_file_path)[0]
    
    with open(f"{base_name}_train.txt", 'w') as f:
        f.writelines(train_lines)
        
    with open(f"{base_name}_val.txt", 'w') as f:
        f.writelines(val_lines)
    
    print(f"{base_name}_train.txt और {base_name}_val.txt बनाए गए।")

# डिटेक्शन और रिकॉग्निशन दोनों के लिए विभाजन बनाएँ
create_splits('dataset/det_label.txt')
create_splits('dataset/rec_label.txt')

अब आपका `dataset` फ़ोल्डर PaddleOCR के साथ ट्रेनिंग के लिए तैयार है!